In [2]:
import pandas as pd
import numpy as np
import duckdb
import os
import glob
from memory_limits import duck_options
from datetime import datetime

In [3]:
mlhd_path = '../data/mlhd/*/*.parquet'
musicbrainz_path = '../data/musicbrainz.db'
mlhd_ids_path = '../data/mlhd_ids.parquet'

data_start = datetime(2005, 12, 31, 23, 59, 59).timestamp()
data_end = datetime(2015, 1, 1, 0, 0, 1).timestamp()
print(data_start)

conn = duckdb.connect(config=duck_options())
brainz_conn = duckdb.connect(musicbrainz_path, config=duck_options())
mlhd = conn.execute(f"select count(*) from read_parquet('{mlhd_path}') where array_length(artist_ids)=1 and timestamp<{data_end} and timestamp>{data_start} ")
# OR timestamp>{data_end} OR timestamp<{data_start}

1136091599.0
-1 -1 236564234240
-1 -1 236564234240


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
count = mlhd.fetchdf()
print(count)
# single artist data = 20,658,034,565
# single artist data 2006-1-1 to 2014-12-31 = 20,462,062,311

   count_star()
0   20462062311


In [5]:
mlhd_conn.execute("SHOW TABLES").fetchall()

[('mlhd_plays',)]

In [4]:
mlhd_conn.execute(f"DESCRIBE SELECT * FROM mlhd_plays").fetchdf()

,column_name,column_type,null,key,default,extra
0,user_id,UUID,YES,None,None,None
1,timestamp,BIGINT,YES,None,None,None
2,artist_ids,UUID[],YES,None,None,None
3,release_id,UUID,YES,None,None,None
4,rec_id,UUID,YES,None,None,None


In [3]:
#check the year
date_query = f"""SELECT EXTRACT(YEAR FROM TO_TIMESTAMP(timestamp / 1000)) AS year,
       COUNT(*) AS row_count
FROM mlhd_plays
GROUP BY year
ORDER BY year; """

# mlhd_conn.execute(date_query).fetchdf()
#output:[(1970, 21079612671)]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,year,row_count
0,1970,20658034565


In [4]:
# check null user or rec id in mlhd data
columns_query = f"""
SELECT COUNT(*)
FROM mlhd_plays
WHERE user_id IS NULL OR rec_id IS NULL LIMIT 5
"""

# mlhd_conn.execute(columns_query).fetchall()
#output: [(0,)]

[(0,)]

In [ ]:
# preprocessing on the whole data

# conn.execute("""
#     DELETE FROM mlhd_plays
#     WHERE user_id IS NULL OR artist_id IS NULL
#     OR strftime('%Y', timestamp) NOT BETWEEN '1970' AND '2024';
# """)

# exclude arists without gender information
del_artist = conn.execute("""
    DELETE FROM mlhd_plays
    WHERE artist_ids[1] NOT IN (
        SELECT artist_id
        FROM '{musicbrainz_path}'.mb_artist
    );
""")

deleted_rows = del_artist.fetchall()[0][0]
print(f"Number of rows deleted for arists without gender information: {deleted_rows}")

In [ ]:
# preprocessing on train

deleted_rows = del_user.fetchall()[0][0]
print(f"Number of rows deleted for users with <5 interactions: {deleted_rows}")

# exclude artists with fewer than 15 total listens across all users
del_artist_unpop = conn.execute("""
    DELETE FROM mlhd_plays
    WHERE artist_ids[1] NOT IN (
        SELECT UNNEST(artist_ids) AS artist_id
        FROM mlhd_plays
        GROUP BY artist_id
        HAVING COUNT(*) >= 15
    );
""")
deleted_rows = del_artist_unpop.fetchall()[0][0]
print(f"Number rows deleted for artists with <15 total listens: {deleted_rows}")

# exclude users with fewer than 15 distinct artists in their data
del_user_artist = conn.execute("""
    DELETE FROM mlhd_plays
    WHERE user_id NOT IN (
        SELECT user_id FROM mlhd_plays,
        UNNEST(artist_ids) AS artist_id
        GROUP BY user_id
        HAVING COUNT(DISTINCT artist_id) >= 15
    );
""")

deleted_rows = del_user_artist.fetchall()[0][0]
print(f"Number of rows deleted for users with <15 artists : {deleted_rows}")

conn.close()

In [ ]:
processed_data = f"""
select(*) from read_parquet('{mlhd_path}') 
where array_length(artist_ids)=1 
and {data_start}<timestamp<{data_end}
left join read_parquet('{mlhd_ids_path}') 
on artist_id
and artist_ids[1] NOT IN (
        SELECT artist_id
        FROM '{musicbrainz_path}'.mb_artist
    );
"""
